# Chapter 19: Training and Deploying TensorFlow Models at Scale

Setelah kita berhasil melatih model yang hebat, langkah selanjutnya adalah membawanya ke tahap produksi. Bab ini membahas aspek-aspek praktis dan teknis untuk menyajikan (*serving*) dan melatih model TensorFlow dalam skala besar.

Topik utama yang akan kita bahas:
* **Menyajikan Model dengan TensorFlow Serving:** Cara mengekspor model ke format `SavedModel` dan men-deploy-nya menggunakan TF Serving, sebuah server model berkinerja tinggi.
* **Men-deploy Model di Google Cloud AI Platform:** Menggunakan layanan cloud untuk menyajikan model kita dengan skalabilitas otomatis.
* **Men-deploy Model ke Perangkat Mobile atau Embedded:** Menggunakan **TensorFlow Lite (TFLite)** untuk mengoptimalkan dan menjalankan model di perangkat dengan sumber daya terbatas.
* **Melatih Model di Banyak Perangkat:** Menggunakan GPU dan **Distribution Strategies API** dari TensorFlow untuk mempercepat pelatihan dengan mendistribusikan komputasi di beberapa GPU atau bahkan beberapa mesin.

## Serving a TensorFlow Model with TensorFlow Serving

Setelah model dilatih, cara terbaik untuk menggunakannya dalam aplikasi produksi adalah dengan membungkusnya dalam sebuah layanan web. **TensorFlow Serving** adalah solusi yang efisien dan teruji untuk ini. Ia dapat menyajikan beberapa model atau versi, dan secara otomatis men-deploy versi terbaru.

### Mengekspor Model ke Format `SavedModel`
Langkah pertama adalah menyimpan model terlatih kita ke dalam format `SavedModel`, format universal TensorFlow untuk serialisasi model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os

# Asumsikan kita punya model Keras yang sudah dilatih
model = keras.models.Sequential([
    keras.layers.Dense(10, input_shape=[8]),
    keras.layers.Dense(1)
])
# model.compile(...) & model.fit(...)

# Menyimpan model ke format SavedModel
model_version = "0001"
model_name = "my_regression_model"
model_path = os.path.join("Chapter 19", model_name, model_version)

# tf.saved_model.save(model, model_path)
# Untuk model Keras, cara yang lebih mudah adalah:
model.save(model_path)

### Menggunakan TensorFlow Serving
Setelah model disimpan, kita bisa menggunakan Docker untuk menjalankan TF Serving dan menyajikan model kita melalui REST API atau gRPC API.

In [ ]:
# Konseptual: Membuat permintaan REST ke TF Serving
import json
import requests
import numpy as np

# Data input dummy
X_new = np.random.rand(3, 8).astype(np.float32)

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

SERVER_URL = 'http://localhost:8501/v1/models/my_regression_model:predict'
# response = requests.post(SERVER_URL, data=input_data_json)
# y_proba = np.array(response.json()["predictions"])

## Deploying a Model to a Mobile or Embedded Device

Untuk perangkat mobile atau embedded, kita perlu model yang ringan dan efisien. **TensorFlow Lite (TFLite)** menyediakan alat untuk ini, dengan tiga tujuan utama:
1. **Mengurangi ukuran model:** Agar unduhan lebih cepat dan penggunaan RAM lebih sedikit.
2. **Mengurangi komputasi:** Untuk latensi yang lebih rendah dan konsumsi baterai yang lebih hemat.
3. **Menyesuaikan model:** Dengan batasan perangkat tertentu.

Ini dicapai melalui teknik seperti **quantization**, di mana bobot model diubah dari float 32-bit menjadi integer 8-bit, mengurangi ukuran model hingga 4x.

In [ ]:
# Mengonversi SavedModel ke format TFLite
saved_model_path = os.path.join("Chapter 19", model_name, model_version)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)

# Mengaktifkan optimisasi (misalnya, quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Menyimpan model TFLite
with open("Chapter 19/converted_model.tflite", "wb") as f:
    f.write(tflite_model)

## Training Models Across Multiple Devices

Pelatihan DNN yang besar bisa sangat lambat. Untuk mempercepatnya, kita bisa mendistribusikan pelatihan di beberapa perangkat (GPU atau TPU).

**Data Parallelism** adalah pendekatan yang paling umum: model direplikasi di setiap perangkat, dan setiap replika dilatih pada *mini-batch* data yang berbeda. Gradien yang dihasilkan kemudian diagregasi untuk memperbarui parameter model.

TensorFlow menyediakan **Distribution Strategies API** untuk melakukan ini dengan mudah. **`MirroredStrategy`** adalah strategi yang paling umum untuk pelatihan di beberapa GPU pada satu mesin.

In [ ]:
# Konseptual: Menggunakan MirroredStrategy
import numpy as np

# Membuat strategi
distribution = tf.distribute.MirroredStrategy()

# Membuat dan mengkompilasi model di dalam scope strategi
with distribution.scope():
    mirrored_model = keras.models.Sequential([
        keras.layers.Dense(10, activation='relu', input_shape=[8]),
        keras.layers.Dense(1)
    ])
    mirrored_model.compile(loss="mse", optimizer="sgd")

# Dummy data untuk pelatihan
X_train = np.random.rand(1000, 8)
y_train = np.random.rand(1000, 1)

# Melatih model seperti biasa
# history = mirrored_model.fit(X_train, y_train, epochs=10)

### Visualisasi Perbandingan Kecepatan Pelatihan

Menggunakan beberapa GPU dapat secara dramatis mengurangi waktu pelatihan, meskipun peningkatannya tidak selalu linear karena adanya overhead komunikasi antar perangkat.

In [ ]:
import matplotlib.pyplot as plt

# Data dummy konseptual untuk perbandingan waktu pelatihan
devices = ['1 GPU', '2 GPUs', '4 GPUs', '8 GPUs']
training_time = [60, 35, 20, 15] # dalam menit

plt.figure(figsize=(8, 5))
plt.bar(devices, training_time, color='skyblue')
plt.xlabel("Jumlah Perangkat")
plt.ylabel("Waktu Pelatihan (menit)")
plt.title("Perbandingan Kecepatan Pelatihan dengan Data Parallelism")
plt.show()